In [1]:
%pip install -q ultralytics kagglehub pyyaml opencv-python efficientnet_pytorch torch torchvision

import os
import shutil
import pathlib
import yaml
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.datasets import ImageFolder

import kagglehub
from ultralytics import YOLO
from efficientnet_pytorch import EfficientNet

#**********************************************
base_p = pathlib.Path("/content") if os.path.isdir("/content") else pathlib.Path.cwd()
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

local_path = pathlib.Path("/content/datasets/crop-pests")
local_path.parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(path, local_path, dirs_exist_ok=True)

data_yaml_path = local_path / "data.yaml"
data_cfg = {
    "path": str(local_path),
    "train": "train/images",
    "val":   "valid/images",
    "test":  "test/images",
    "names":
     [
      "Ants",
      "Bees",
      "Beetles",
      "Catterpillars",
      "Earthworms",
      "Earwigs",
      "Grasshoppers",
      "Moths",
      "Slugs",
      "Snails",
      "Wasps",
      "Weevils",
    ]
}

with open(data_yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)
#**********************************************


cfg_p = data_yaml_path
runs_p = base_p / "yolo_runs"
runs_p.mkdir(parents=True, exist_ok=True)


yolo_1 = YOLO("yolov8n.pt")

yolo_1.train(
    data=str(cfg_p),
    epochs=10,
    imgsz=640,
    batch=8,
    device=0,
    workers=2,
    project=str(runs_p),
    name="cp_s1_mosaic",
    verbose=False,
    mosaic=0.5,
    mixup=0.0,
    copy_paste=0.0,
)
#**********************************************

v1 = yolo_1.val()

box1 = v1.box
names1 = v1.names

m1_map = float(box1.map)
m1_prec = float(box1.mp)
m1_rec = float(box1.mr)

per_class_map1 = {}
if box1.maps is not None:
    for cid, cname in names1.items():
        if cid < len(box1.maps):
            per_class_map1[cname] = float(box1.maps[cid])

#**********************************************

best_1 = runs_p / "cp_s1_mosaic" / "weights" / "best.pt"

yolo_2 = YOLO(str(best_1))

yolo_2.train(
    data=str(cfg_p),
    epochs=5,
    imgsz=640,
    batch=8,
    device=0,
    workers=2,
    project=str(runs_p),
    name="cp_s2_nomosaic",
    verbose=False,
    mosaic=0.0,
    mixup=0.0,
    copy_paste=0.0,
)

v2 = yolo_2.val()

box2 = v2.box
names2 = v2.names

m2_map  = float(box2.map)
m2_prec = float(box2.mp)
m2_rec  = float(box2.mr)

per_class_map2 = {}
if box2.maps is not None:
    for cid, cname in names2.items():
        if cid < len(box2.maps):
            per_class_map2[cname] = float(box2.maps[cid])

#**********************************************

print("Stage 1 Results")
print("Overall:", {
    "mAP50-95(B)": round(m1_map, 4),
    "precision(B)": round(m1_prec, 4),
    "recall(B)": round(m1_rec, 4),
})
print("Per-class mAP50-95:")
for cls, score in per_class_map1.items():
    print(f"  {cls}: {score:.4f}")

print("Stage 2 Results")
print("Overall:", {
    "mAP50-95(B)": round(m2_map, 4),
    "precision(B)": round(m2_prec, 4),
    "recall(B)": round(m2_rec, 4),
})

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Using Colab cache for faster access to the 'crop-pests-dataset' dataset.
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/crop-pests/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, 